In [2]:
import math
import pulp
import numpy as np

In [3]:
num_nodes = 32
capacity = 100
demand = np.array([0, 19, 21, 6, 19, 7, 12, 16, 6, 16, 
                   8, 14, 21, 16, 3, 22, 18, 19, 1, 24,
                   8, 12, 4, 8, 24, 24, 2, 20, 15, 2,
                   14, 9])

In [4]:
coordinate = np.array([[82, 76],
 [96, 44],
 [50, 5],
 [49, 8],
 [13, 7],
 [29, 89],
 [58, 30],
 [84, 39],
 [14, 24],
 [2, 39],
 [3, 82],
 [5, 10],
 [98, 52],
 [84, 25],
 [61, 59],
 [1, 65],
 [88, 51],
 [91, 2],
 [19, 32],
 [93, 3],
 [50, 93],
 [98, 14],
 [5, 42],
 [42, 9],
 [61, 62],
 [9, 97],
 [80, 55],
 [57, 69],
 [23, 15],
 [20, 70],
 [85, 60],
 [98, 5]])


In [5]:
coordinate[10]

array([ 3, 82])

In [6]:
def computeDistance(c1, c2):
    return math.sqrt( pow(c2[0] - c1[0], 2) + pow(c2[1] - c1[1], 2))
distance = np.array([ [ round(computeDistance( c1, c2 )) for c1 in coordinate ] \
                    for c2 in coordinate ])

In [7]:
distance

array([[  0,  35,  78, ...,  62,  16,  73],
       [ 35,   0,  60, ...,  80,  19,  39],
       [ 78,  60,   0, ...,  72,  65,  48],
       ...,
       [ 62,  80,  72, ...,   0,  66, 102],
       [ 16,  19,  65, ...,  66,   0,  57],
       [ 73,  39,  48, ..., 102,  57,   0]])

In [8]:
# formulate CVRP as integer programming problem
# V = { 0, 1, ..., num_nodes - 1 }, where 0 means the depot
problem = pulp.LpProblem( "CVRP", pulp.LpMinimize )

In [1]:
## variables
x = np.array([ [ pulp.LpVariable( 'x_{}_{}'.format( i, j ), cat="Binary" ) \
        if i != j else None for j in range(num_nodes) ] \
        for i in range(num_nodes) ])
u = np.array([ pulp.LpVariable( 'u_{}'.format( i ), demand[i], capacity, cat="Integer" ) \
        for i in range(1,num_nodes) ])

NameError: name 'np' is not defined

In [65]:
## objective function: \sum_{i \in V}\sum_{j \in V, i!=j} c_ij x_ij
problem += pulp.lpSum( distance[i][j] * x[i][j] for i in range(num_nodes) \
                        for j in range(num_nodes) if i != j )


In [66]:
## constraints:
### \sum_{i \in V\{j}} x_ij = 1 for all j \in V\{0}
for j in range(1,num_nodes):
    problem += pulp.lpSum( x[i][j] for i in range(num_nodes) if i != j ) == 1

### \sum_{j \in V\{i}} x_ij = 1 for all i \in V\{0}
for i in range(1,num_nodes):
    problem += pulp.lpSum( x[i][j] for j in range(num_nodes) if i != j ) == 1

### u_i - u_j + C x_ij <= C - d_j for i, j \in V\{0} i != j such that d_i  + d_j <= C
for i in range(1,num_nodes):
    for j in range(1,num_nodes):
        if i != j and demand[i] + demand[j] <= capacity:
            problem += u[i-1] - u[j-1] + capacity * x[i][j] <= capacity - demand[j]


In [67]:
# solve
threads = 8
maxSeconds = 60
result = problem.solve(pulp.PULP_CBC_CMD(threads=threads, maxSeconds=maxSeconds))

print("objective value = {}".format(pulp.value(problem.objective)))


objective value = 1274.0
